In [0]:
# Databricks notebook source
# ============================================================================
# SECTION 1: NOTEBOOK CONFIGURATION AND SETUP
# ============================================================================
# This section initializes the notebook environment and sets up necessary
# configurations for the ETL pipeline

# Import required libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, trim, upper, lower, when, count, sum, avg, 
    max, min, round, to_date, year, month, current_timestamp,
    regexp_replace, coalesce, isnan, lit, countDistinct
)
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType, 
    DoubleType, DateType, TimestampType
)
from pyspark.sql.window import Window
from datetime import datetime

# Display notebook configuration
print("=" * 80)
print("ETL PIPELINE - RAW DATA TO UNITY CATALOG")
print("=" * 80)
print(f"Spark Version: {spark.version}")
print(f"Execution Time: {datetime.now()}")

In [0]:
# ============================================================================
# SECTION 2: UNITY CATALOG CONFIGURATION
# ============================================================================
# Define the three-level namespace for Unity Catalog: catalog.schema.table
# Unity Catalog provides centralized governance for all data assets

# Define Notebook Parameters
dbutils.widgets.text("CATALOG_NAME", "workspace")
dbutils.widgets.text("SCHEMA_NAME", "portfolio_project")
dbutils.widgets.text("TABLE_NAME", "sales_summary")

# Define Unity Catalog namespace variables
CATALOG_NAME = dbutils.widgets.get("CATALOG_NAME")
SCHEMA_NAME = dbutils.widgets.get("SCHEMA_NAME")
TABLE_NAME = dbutils.widgets.get("TABLE_NAME")
FULL_TABLE_PATH = f"{CATALOG_NAME}.{SCHEMA_NAME}.{TABLE_NAME}"

# Raw data paths
RAW_CSV_PATH = "/Volumes/workspace/portfolio_projects/volume_portfolio_projects/simple_etl_project_raw_data_csv/"
RAW_PARQUET_PATH = "/Volumes/workspace/portfolio_projects/volume_portfolio_projects/simple_etl_project_raw_data_parquet/"

print(f"Unity Catalog Configuration:")
print(f"  - Catalog: {CATALOG_NAME}")
print(f"  - Schema: {SCHEMA_NAME}")
print(f"  - Table: {TABLE_NAME}")
print(f"  - Full Path: {FULL_TABLE_PATH}")

In [0]:
# ============================================================================
# SECTION 3: CREATE CATALOG AND SCHEMA
# ============================================================================
# Set up the Unity Catalog infrastructure if it doesn't exist
# This ensures proper organization and governance of data assets

# Activate the target catalog
spark.sql(f"USE CATALOG {CATALOG_NAME}")
print(f"✓ Using catalog: {CATALOG_NAME}")

# Create schema if it doesn't exist
spark.sql(f"""
    CREATE SCHEMA IF NOT EXISTS {CATALOG_NAME}.{SCHEMA_NAME}
    COMMENT 'Portfolio project schema for ETL demonstration'
""")
print(f"✓ Schema created/verified: {SCHEMA_NAME}")

# Verify the schema was created successfully
schemas_df = spark.sql(f"SHOW SCHEMAS IN {CATALOG_NAME}")
display(schemas_df)

In [0]:
# ============================================================================
# SECTION 4: EXTRACT - LOAD RAW DATA
# ============================================================================
# Extract raw data from various file formats
# Databricks supports CSV, Parquet, JSON, Delta, and more

# Option 1: Load data from CSV
print("Loading data from CSV...")
df_raw = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("dateFormat", "yyyy-MM-dd") \
    .option("nullValue", "NULL") \
    .option("mode", "PERMISSIVE") \
    .load(RAW_CSV_PATH)

# Option 2: Load data from Parquet (more efficient for large datasets)
# Uncomment the following lines to use Parquet instead
# print("Loading data from Parquet...")
# df_raw = spark.read \
#     .format("parquet") \
#     .load(RAW_PARQUET_PATH)

# Option 3: Load from Delta Lake (recommended for Databricks)
# df_raw = spark.read.format("delta").load("/path/to/delta/table")

print(f"✓ Raw data loaded successfully")
print(f"  - Total rows: {df_raw.count():,}")
print(f"  - Total columns: {len(df_raw.columns)}")

# Display sample of raw data
print("\nSample of raw data (first 10 rows):")
display(df_raw.limit(10))

In [0]:
# ============================================================================
# SECTION 5: DATA PROFILING AND QUALITY ASSESSMENT
# ============================================================================
# Before cleaning, it's crucial to understand data quality issues
# This section provides comprehensive data profiling

print("=" * 80)
print("DATA QUALITY ASSESSMENT")
print("=" * 80)

# Check for null values in each column
print("\n1. NULL VALUE ANALYSIS:")
null_counts = df_raw.select([
    count(when(col(c).isNull(), c)).alias(c) 
    for c in df_raw.columns
])
display(null_counts)

# Check for duplicate records
duplicate_count = df_raw.count() - df_raw.dropDuplicates().count()
print(f"\n2. DUPLICATE RECORDS: {duplicate_count:,}")

# Basic statistical summary
print("\n3. STATISTICAL SUMMARY:")
display(df_raw.describe())

# Check data types
print("\n4. CURRENT DATA TYPES:")
df_raw.printSchema()

In [0]:
# ============================================================================
# SECTION 6: TRANSFORM - DATA CLEANING
# ============================================================================
# Apply comprehensive data cleaning operations
# Each step is explained and can be customized based on your data

print("Starting data cleaning process...")

# Step 1: Remove exact duplicate rows
df_cleaned = df_raw.dropDuplicates()
print(f"✓ Step 1: Removed {df_raw.count() - df_cleaned.count()} duplicate rows")

# Step 2: Handle missing values with business logic
# Different strategies for different columns
df_cleaned = df_cleaned \
    .na.fill({
        "quantity": 0,          # Fill missing quantities with 0
        "unit_price": 0.0,      # Fill missing prices with 0
        "discount": 0.0,        # Assume no discount if missing
        "category": "Unknown",  # Default category
        "region": "Unspecified" # Default region
    })
print("✓ Step 2: Filled missing values with appropriate defaults")

# Step 3: Drop rows where critical fields are null
# Transaction ID and Date are mandatory for our analysis
df_cleaned = df_cleaned \
    .filter(col("transaction_id").isNotNull()) \
    .filter(col("transaction_date").isNotNull()) \
    .filter(col("customer_id").isNotNull())
print("✓ Step 3: Removed rows with null critical fields")

# Step 4: Standardize text fields (trim whitespace, consistent casing)
df_cleaned = df_cleaned \
    .withColumn("customer_name", trim(upper(col("customer_name")))) \
    .withColumn("product_name", trim(upper(col("product_name")))) \
    .withColumn("category", trim(upper(col("category")))) \
    .withColumn("region", trim(upper(col("region")))) \
    .withColumn("sales_person", trim(upper(col("sales_person"))))
print("✓ Step 4: Standardized text fields (uppercase and trimmed)")

# Step 5: Convert date strings to proper date type
df_cleaned = df_cleaned \
    .withColumn("transaction_date", to_date(col("transaction_date"), "yyyy-MM-dd"))
print("✓ Step 5: Converted transaction_date to proper DateType")

# Step 6: Add derived columns for better analysis
df_cleaned = df_cleaned \
    .withColumn("year", year(col("transaction_date"))) \
    .withColumn("month", month(col("transaction_date"))) \
    .withColumn("total_amount", 
                round((col("quantity") * col("unit_price")) * (1 - col("discount")), 2))
print("✓ Step 6: Added derived columns (year, month, total_amount)")

# Step 7: Data validation - remove invalid records
# Ensure quantity and price are positive
df_cleaned = df_cleaned \
    .filter(col("quantity") >= 0) \
    .filter(col("unit_price") >= 0) \
    .filter(col("discount") >= 0) \
    .filter(col("discount") <= 1)  # Discount should be between 0 and 1
print("✓ Step 7: Filtered out invalid records (negative values, invalid discounts)")

# Step 8: Remove outliers (optional - adjust thresholds based on your data)
# For example, remove transactions with unrealistic quantities
df_cleaned = df_cleaned.filter(col("quantity") <= 1000)
print("✓ Step 8: Removed outliers (quantity > 1000)")

print(f"\nCleaning complete:")
print(f"  - Original rows: {df_raw.count():,}")
print(f"  - Cleaned rows: {df_cleaned.count():,}")
print(f"  - Rows removed: {df_raw.count() - df_cleaned.count():,}")
print(f"  - Data retention rate: {(df_cleaned.count() / df_raw.count() * 100):.2f}%")

# Display sample of cleaned data
print("\nSample of cleaned data:")
display(df_cleaned.limit(10))

In [0]:
# ============================================================================
# SECTION 7: TRANSFORM - DATA AGGREGATION
# ============================================================================
# Create aggregated views of the data for analytical purposes
# This demonstrates various aggregation techniques in PySpark

print("Creating aggregated dataset...")

# Aggregation 1: Monthly sales summary by category and region
df_aggregated = df_cleaned.groupBy(
    "year",
    "month",
    "category",
    "region"
).agg(
    count("transaction_id").alias("total_transactions"),
    sum("quantity").alias("total_quantity_sold"),
    sum("total_amount").alias("total_revenue"),
    avg("total_amount").alias("avg_transaction_value"),
    max("total_amount").alias("max_transaction_value"),
    min("total_amount").alias("min_transaction_value"),
    countDistinct("customer_id").alias("unique_customers"),
    countDistinct("product_id").alias("unique_products")
).orderBy("year", "month", "category", "region")

# Round decimal values for better readability
df_aggregated = df_aggregated \
    .withColumn("total_revenue", round(col("total_revenue"), 2)) \
    .withColumn("avg_transaction_value", round(col("avg_transaction_value"), 2)) \
    .withColumn("max_transaction_value", round(col("max_transaction_value"), 2)) \
    .withColumn("min_transaction_value", round(col("min_transaction_value"), 2))

# Add metadata columns for tracking
df_aggregated = df_aggregated \
    .withColumn("created_at", current_timestamp()) \
    .withColumn("data_source", lit("sales_etl_pipeline"))

print(f"✓ Aggregation complete")
print(f"  - Aggregated rows: {df_aggregated.count():,}")
print(f"  - Aggregation dimensions: year, month, category, region")
print(f"  - Metrics calculated: 8 business metrics")

# Display aggregated data
print("\nSample of aggregated data:")
display(df_aggregated.limit(20))

# Show aggregation summary
print("\nAggregation summary by category:")
display(
    df_aggregated.groupBy("category")
    .agg(
        sum("total_revenue").alias("category_revenue"),
        sum("total_transactions").alias("category_transactions")
    )
    .orderBy(col("category_revenue").desc())
)

In [0]:
# ============================================================================
# SECTION 8: LOAD - WRITE TO UNITY CATALOG
# ============================================================================
# Save the cleaned and aggregated data to Unity Catalog
# Using Delta Lake format for ACID transactions and time travel

print(f"Writing data to Unity Catalog...")
print(f"Target: {FULL_TABLE_PATH}")

# Write the aggregated data to Unity Catalog as a managed table
# Mode options: "overwrite", "append", "error", "ignore"
df_aggregated.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .option("delta.columnMapping.mode", "name") \
    .saveAsTable(FULL_TABLE_PATH)

print(f"✓ Data successfully written to Unity Catalog")

# Add table comment and properties
spark.sql(f"""
    COMMENT ON TABLE {FULL_TABLE_PATH} IS 
    'Aggregated sales data by month, category, and region. 
    Source: ETL pipeline from raw sales data. 
    Updated: {datetime.now()}'
""")

# Optimize the table for better query performance
spark.sql(f"OPTIMIZE {FULL_TABLE_PATH}")
print("✓ Table optimized for query performance")

# Analyze table statistics for the query optimizer
spark.sql(f"ANALYZE TABLE {FULL_TABLE_PATH} COMPUTE STATISTICS FOR ALL COLUMNS")
print("✓ Table statistics computed")

In [0]:
# ============================================================================
# SECTION 9: DATA VALIDATION AND QUALITY CHECKS
# ============================================================================
# Verify the data was written correctly and perform quality checks

print("=" * 80)
print("POST-LOAD VALIDATION")
print("=" * 80)

# Check 1: Verify table exists and count rows
table_count = spark.sql(f"SELECT COUNT(*) as row_count FROM {FULL_TABLE_PATH}").collect()[0][0]
print(f"\n✓ Table exists with {table_count:,} rows")

# Check 2: Display table schema
print("\nTable schema:")
spark.sql(f"DESCRIBE TABLE {FULL_TABLE_PATH}").show(truncate=False)

# Check 3: Display sample data from Unity Catalog
print("\nSample data from Unity Catalog table:")
display(spark.sql(f"SELECT * FROM {FULL_TABLE_PATH} LIMIT 10"))

# Check 4: Run basic analytics query
print("\nTop 5 categories by revenue:")
top_categories = spark.sql(f"""
    SELECT 
        category,
        SUM(total_revenue) as total_revenue,
        SUM(total_transactions) as total_transactions,
        SUM(unique_customers) as total_customers
    FROM {FULL_TABLE_PATH}
    GROUP BY category
    ORDER BY total_revenue DESC
    LIMIT 5
""")
display(top_categories)

# Check 5: Verify data integrity
print("\nData integrity checks:")
integrity_check = spark.sql(f"""
    SELECT 
        COUNT(*) as total_rows,
        COUNT(DISTINCT year, month, category, region) as unique_combinations,
        SUM(CASE WHEN total_revenue < 0 THEN 1 ELSE 0 END) as negative_revenue_count,
        SUM(CASE WHEN total_transactions < 0 THEN 1 ELSE 0 END) as negative_transaction_count
    FROM {FULL_TABLE_PATH}
""")
display(integrity_check)

In [0]:
# ============================================================================
# FINAL SUMMARY
# ============================================================================
print("=" * 80)
print("ETL PIPELINE COMPLETED SUCCESSFULLY")
print("=" * 80)
print(f"\n📊 Summary:")
print(f"  • Source: Raw CSV/Parquet files")
print(f"  • Destination: {FULL_TABLE_PATH}")
print(f"  • Records processed: {df_raw.count():,}")
print(f"  • Records loaded: {table_count:,}")
print(f"  • Data quality: ✓ Cleaned and validated")
print(f"  • Format: Delta Lake")
print(f"  • Governance: Unity Catalog")
print(f"\n✅ Data is now available for analytics and reporting")
print(f"\nNext steps:")
print(f"  1. Query the table: SELECT * FROM {FULL_TABLE_PATH}")
print(f"  2. Create dashboards using Databricks SQL")
print(f"  3. Set up automated jobs for regular updates")
print(f"  4. Configure alerts for data quality monitoring")